In [12]:
import matplotlib.pyplot as plt

In [144]:
import os
import cv2
import torch
import numpy as np
from google.colab import drive
from PIL import Image
from collections import defaultdict
from IPython.display import clear_output 

In [2]:
%cd /content
!git clone https://github.com/ultralytics/yolov3

%cd yolov3
%pip install -qr requirements.txt 

print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

/content
Cloning into 'yolov3'...
remote: Enumerating objects: 9727, done.
remote: Total 9727 (delta 0), reused 0 (delta 0), pack-reused 9727
Receiving objects: 100% (9727/9727), 8.24 MiB | 32.43 MiB/s, done.
Resolving deltas: 100% (6632/6632), done.
/content/yolov3
     |████████████████████████████████| 645kB 25.5MB/s 
Setup complete. Using torch 1.8.1+cu101 _CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15109MB, multi_processor_count=40)


In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!mkdir images
!mkdir output
!gdown --id 1c4AOUv43CGOH3qR9axkN7J30s0TlAHbT -O /content/video.mp4

Downloading...
From: https://drive.google.com/uc?id=1c4AOUv43CGOH3qR9axkN7J30s0TlAHbT
To: /content/video.mp4
14.6MB [00:00, 46.5MB/s]


In [5]:
video = cv2.VideoCapture('/content/video.mp4');
fps = video.get(cv2.CAP_PROP_FPS)
video.release(); 
fps

30.0

In [6]:
!ffmpeg -i '/content/video.mp4' '/content/yolov3/images/out-%06d.jpg'
clear_output()

In [7]:
%%time
!python detect.py --weights /content/drive/MyDrive/Alex.pt --img 640 --source /content/yolov3/images/ --save-txt

Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres=0.25, device='', exist_ok=False, img_size=640, iou_thres=0.45, name='exp', project='runs/detect', save_conf=False, save_txt=True, source='/content/yolov3/images/', update=False, view_img=False, weights=['/content/drive/MyDrive/Alex.pt'])
YOLOv3 v9.1-7-gc1f8dd9 torch 1.8.1+cu101 CUDA:0 (Tesla T4, 15109.75MB)

Fusing layers... 
Model Summary: 261 layers, 61922845 parameters, 0 gradients, 156.3 GFLOPS
image 1/1203 /content/yolov3/images/out-000001.jpg: 384x640 2 traffic lights, Done. (0.054s)
image 2/1203 /content/yolov3/images/out-000002.jpg: 384x640 1 traffic lights, Done. (0.024s)
image 3/1203 /content/yolov3/images/out-000003.jpg: 384x640 1 traffic lights, Done. (0.024s)
image 4/1203 /content/yolov3/images/out-000004.jpg: 384x640 1 traffic lights, Done. (0.024s)
image 5/1203 /content/yolov3/images/out-000005.jpg: 384x640 1 traffic lights, Done. (0.024s)
image 6/1203 /content/yolov3/images/out-000006.jpg: 384x640 1 t

In [15]:
def detect_color(img, Threshold=0.01):
    img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # lower mask (0-10)
    lower_red = np.array([0, 70, 50])
    upper_red = np.array([10, 255, 255])
    mask0 = cv2.inRange(img_hsv, lower_red, upper_red)

    # upper mask (170-180)
    lower_red1 = np.array([170, 70, 50])
    upper_red1 = np.array([180, 255, 255])
    mask1 = cv2.inRange(img_hsv, lower_red1, upper_red1)

    # defining the Range of yellow color
    lower_yellow = np.array([21, 39, 64])
    upper_yellow = np.array([40, 255, 255])
    mask2 = cv2.inRange(img_hsv, lower_yellow, upper_yellow)

    # defining the Range of green color
    lowew_green = np.array([41, 52, 72])
    upper_green = np.array([102, 255, 255])
    mask3 = cv2.inRange(img_hsv, lowew_green, upper_green)

    # red pixels' mask
    mask_red = mask0 + mask1
    mask_yellow = mask2
    mask_green = mask3

    # Compare the percentage
    rate_red = np.count_nonzero(mask_red) / (img.shape[0] * img.shape[1])
    rate_yellow = np.count_nonzero(mask_yellow) / (img.shape[0] * img.shape[1])
    rate_green = np.count_nonzero(mask_green) / (img.shape[0] * img.shape[1])
    rates = [rate_red, rate_yellow, rate_green]
    colors = ['red', 'yellow', 'green']
    rate_max = max(rates)

    if rate_max > Threshold:
        return colors[rates.index(rate_max)]
    else:
        return 'unknown'

In [152]:
colors = defaultdict(list)
img_folder = '/content/yolov3/images'
label_folder = '/content/yolov3/runs/detect/exp/labels'
label_names = os.listdir(label_folder)
label_names.sort()

In [155]:
for label_name in label_names:
    label_path = os.path.join(label_folder, label_name)
    with open(label_path, 'r') as f:
        lines = f.readlines()
    for line in lines:
        xywh = np.array([float(s) for s in line.split(' ')][1:])
        img_name = os.path.splitext(label_name)[0] + '.jpg'
        img_path = os.path.join(img_folder, img_name)
        img = cv2.imread(img_path)
        x1 = round((xywh[0] - xywh[2] / 2) * 1280) # !!!
        x2 = round((xywh[0] + xywh[2] / 2) * 1280) # !!!
        y1 = round((xywh[1] - xywh[3] / 2) * 720) # !!!
        y2 = round((xywh[1] + xywh[3] / 2) * 720) # !!!
        img_crop = img[y1:y2,x1:x2]
        color = detect_color(img_crop)
        cv2.putText(img, color, (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 1)
        cv2.imwrite(img_path, img)
        colors[img_name].append(color)

In [150]:
# img_folder = '/content/yolov3/content/yolov3/runs/detect/exp'
# for label_name in label_names:
#     label_path = os.path.join(label_folder, label_name)
#     with open(label_path, 'r') as f:
#         lines = f.readlines()
#     for line in lines:
#         xywh = np.array([float(s) for s in line.split(' ')][1:])
#         img_name = os.path.splitext(label_name)[0] + '.jpg'
#         img_path = os.path.join(img_folder, img_name)
#         img = cv2.imread(img_path)
#         x1 = round((xywh[0] - xywh[2] / 2) * 1280) # !!!
#         y1 = round((xywh[1] - xywh[3] / 2) * 720) # !!!
#         color = next(iter(colors[img_name]))
#         cv2.putText(img, color, (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 1)

In [156]:
!ffmpeg -framerate 30 -i /content/yolov3/images/out-%06d.jpg -c:v libx264 -crf 18 -pix_fmt yuv420p /content/drive/MyDrive/result.mp4

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

In [158]:
!ffmpeg -framerate 30 -i /content/yolov3/images/out-%06d.jpg -c:v libx264 -crf 18 -preset ultrafast -pix_fmt yuv420p /content/drive/MyDrive/result.mp4

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

In [ ]:
!ffmpeg -framerate 30 -i /content/yolov3/runs/detect/exp/out-%06d.jpg -c:v libx264 -crf 18 -pix_fmt yuv420p /content/drive/MyDrive/result.mp4